In [49]:
import json
import pandas as pd
from scr.mymodule.database import get_db_connection
from scr.mymodule.processing import insert_json_data, process_files_in_folder

In [ ]:
# Create a cursor object to execute SQL queries
conn = get_db_connection("scr/config.ini")
cur = conn.cursor()
cur.execute("SELECT * FROM l0_comments")
cur.close()
conn.close()

In [ ]:
file_name = 'data/Love_Quote.json'

# Create a cursor object to execute SQL queries
conn = get_db_connection("scr/config.ini")
insert_json_data(file_name, conn)

In [ ]:
# input_folder = "/Volumes/extra-boss/thesis/raw_data"
# output_folder = "/Volumes/extra-boss/thesis/processed"
# conn = get_db_connection("scr/config.ini")
# process_files_in_folder(input_folder, output_folder, conn)

# Upload data for bot and moderators
Adding data related to bot and moderators in there seprate tables

In [22]:
import pandas as pd

def convert_json_to_csv(json_path, csv_path):
    """Converts a JSON file with bot names to a CSV file with bot names.
    
    Args:
        json_path (str): The path to the JSON file containing bot names.
        csv_path (str): The path to the CSV file to save bot names to.
        
    Returns:
        None.
    """
    # Read the JSON file into a DataFrame
    df = pd.read_json(json_path, lines=True)

    # Transpose the DataFrame, reset the index, and rename the index column to "Name"
    df = df.transpose().reset_index().rename(columns={"index": "Name"})

    # Save only the "Name" column to a CSV file without the index column
    df['Name'].to_csv(csv_path, index=False)

In [23]:
# Call the function with the path to the JSON file and the path to the CSV file
convert_json_to_csv('data/bots.json', 'data/bots.csv')

## Reading the file we created in above process

In [24]:
def print_dataframe_info(df):
    """
    Prints information about the columns of a DataFrame.
    
    Args:
        df: A pandas DataFrame object
        
    Returns:
        None
    """
    # Print the number of columns in the DataFrame
    print(f'Number of columns: {len(df.columns)}')

    # Print the names of the columns in the DataFrame
    print(f'Column names: {", ".join(df.columns)}')

    # Print the data type of each column in the DataFrame
    print('Data types:')
    print(df.dtypes)

    # Print summary statistics for each column in the DataFrame
    print('Summary statistics:')
    print(df.describe())

In [5]:
bots = pd.read_csv('data/bots.csv')

In [25]:
print_dataframe_info(bots)

Number of columns: 1
Column names: names
Data types:
names    object
dtype: object
Summary statistics:
                     names
count                63472
unique               63472
top     KickOpenTheDoorBot
freq                     1


Load data into bots postgress database

In [20]:
# createing list to add to database
bot_names = []
for index, row in bots.iterrows():
    bot_names.append(str(row['names']))


# Create a cursor object to execute SQL queries
conn = get_db_connection("scr/config.ini")
cur = conn.cursor()
insert_post_query =  'INSERT INTO bots (name) VALUES (%s)'
cur.executemany(insert_post_query, [(name,) for name in bot_names])
conn.commit()
cur.close()
conn.close()

In [26]:
# url_moderators = 'https://www.reddit.com/r/MomForAMinute/about/moderators.json'
subreddit = 'MomForAMinute'
url_moderators = f'https://www.reddit.com/r/{subreddit}/about/moderators.json'
print(url_moderators)

https://www.reddit.com/r/MomForAMinute/about/moderators.json


# get moderators

In [42]:
import pandas as pd

def process_json_to_df(json_data):
    data_list = []
    for item in json_data:
        subreddit = item['subreddit']
        for child in item['data']['data']['children']:
            data = [subreddit, child['name'], child['author_flair_text'], child['date'], child['rel_id'], child['id'], child['author_flair_css_class']]
            data_list.append(data)
    df = pd.DataFrame(data_list, columns=["subreddit", "name", "author_flair_text", "date", "rel_id", "id", "author_flair_css_class"])
    return df


In [58]:
import json

# read the JSON data from file
with open('data/list_of_moderators.json') as f:
    json_data = json.load(f)

# process the JSON data to pandas dataframe
df = process_json_to_df(json_data)

# print shape of the dataframe
print(df.shape)

# print the dataframe
df.head()

(32, 7)


,subreddit,name,author_flair_text,date,rel_id,id,author_flair_css_class
0,MomForAMinute,closingbelle,Mother Goose Mod,1.542317e+09,rb_12k6r1s,t2_16shrg,z
1,MomForAMinute,M4aMmod,Moderator,1.664659e+09,rb_4798gpo,t2_szqyvv40,None
2,MomForAMinute,Jaz_the_Nagai,Cool Big Bro,1.672511e+09,rb_4hlfg1z,t2_iu6fw,w
3,MomForAMinute,Chengweiyingji,Duckling,1.672511e+09,rb_4hlfmiu,t2_qx07f,y
4,GetMotivated,carlinha1289,your lil' ray of sunshine,1.404412e+09,rb_6boo3i,t2_5i7ml,S1


In [48]:
# find the maximum length of characters in each column
max_len = df.applymap(lambda x: len(str(x))).max()

# print the result
print(max_len)

subreddit                 17
name                      17
author_flair_text         25
date                      12
rel_id                    10
id                        11
author_flair_css_class     7
dtype: int64


In [54]:
def insert_data(table_name, column_names, data):
    """
    Inserts data into a PostgreSQL database table.

    Args:
        table_name (str): Name of the table to insert data into.
        column_names (list): List of column names in the table to insert data into.
        data (list): List of tuples containing data to insert. Each tuple corresponds to a row in the table.

    Returns:
        None
    """

    # Create a connection to the database
    conn = get_db_connection("scr/config.ini")
    cur = conn.cursor()

    # Create the insert query using parameterized queries
    insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s']*len(column_names))})"

    # Execute the insert query
    cur.executemany(insert_query, data)
    conn.commit()
    cur.close()
    conn.close()

In [55]:
def dataframe_to_tuples(df):
    """Converts a Pandas DataFrame to a list of tuples.

    Args:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        A tuple containing a list of column names and a list of tuples representing the data.
    """
    columns = list(df.columns)
    data = [tuple(row) for row in df.values.tolist()]

    return columns, data

In [56]:
# Convert the DataFrame to a list of tuples
column_names, data = dataframe_to_tuples(df)

# Print the column names and data
print(column_names)
print(data)

['subreddit', 'name', 'author_flair_text', 'date', 'rel_id', 'id', 'author_flair_css_class']
[('MomForAMinute', 'closingbelle', 'Mother Goose Mod', 1542317451.0, 'rb_12k6r1s', 't2_16shrg', 'z'), ('MomForAMinute', 'M4aMmod', 'Moderator', 1664659377.0, 'rb_4798gpo', 't2_szqyvv40', None), ('MomForAMinute', 'Jaz_the_Nagai', 'Cool Big Bro', 1672510982.0, 'rb_4hlfg1z', 't2_iu6fw', 'w'), ('MomForAMinute', 'Chengweiyingji', 'Duckling', 1672511088.0, 'rb_4hlfmiu', 't2_qx07f', 'y'), ('GetMotivated', 'carlinha1289', "your lil' ray of sunshine", 1404411814.0, 'rb_6boo3i', 't2_5i7ml', 'S1'), ('GetMotivated', 'mungoflago', 'Mod', 1455845973.0, 'rb_eamgyj', 't2_9vgfz', 'blank'), ('GetMotivated', 'AutoModerator', None, 1493312605.0, 'rb_o4zdnx', 't2_6l4z3', None), ('GetMotivated', 'SEO_Nuke', None, 1493312626.0, 'rb_o4zdyy', 't2_unwh5', None), ('GetMotivated', 'davidreiss666', 'Mod', 1569545427.0, 'rb_1ivyc2v', 't2_pfn1', 'blank'), ('GetMotivated', 'BotDefense', None, 1650922811.0, 'rb_3pxyqnt', 't2_5

In [57]:
table_name = "subreddit_moderators_l0"
insert_data(table_name, column_names, data)